# 特征工程：共线性、降维、扩展

### 1. 共线性

##### 特征间共线性：

- 两个或多个特征包含了**相似**的信息，相互之间存在**强烈**的相关关系

##### 常用判断标准：

- 两个或两个以上的特征间的相关性系数高于**0.8**。

##### 共线性的影响：

- 降低运算效率

- 降低一些模型的稳定性

- 弱化一些模型的预测能力

##### 处理方式：

- 删除：一组相互共线的特征中**只保留**与y相关性最高的一个
- 变换：对共线的两列特征进行求比值、求差值等计算

In [ ]:
# 特征工程的演示
import pandas as pd
import matplotlib.pyplot as plt
# 样例数据读取
df = pd.read_excel('realestate_sample_preprocessed.xlsx')
# 数据集基本情况查看
print(df.shape)
print(df.dtypes)
print(df.isnull().sum())
print(df.head())

In [ ]:
# 变量相关性矩阵
correlation_table = df.drop(columns='id').corr()
print(correlation_table)

In [ ]:
# 对相关性矩阵进行可视化
import seaborn as sns
# 绘制相关性矩阵热力图
sns.heatmap(correlation_table)
plt.show()

In [ ]:
# 查看共线的日间、夜间、夜间20-39岁人口与房价的关系
print(df[['average_price','daypop','nightpop','night20-39']].corr())

In [ ]:
# 根据共线性矩阵，保留与房价相关性最高的日间人口，将夜间人口和20-39岁夜间人口进行比例处理
def age_percent(row):
    if row['nightpop'] == 0:
        return 0
    else:
        return row['night20-39']/row['nightpop']
df['per_a20_39'] = df.apply(age_percent,axis=1)
df = df.drop(columns=['nightpop','night20-39'])
# 再次查看相关性矩阵的热力图
correlation_table = df.corr()
sns.heatmap(correlation_table)

### 2. 数据降维与特征提取

##### 处理目的：

- 降低不相关特征对于模型准确性的干扰

- 降低模型复杂度，提高模型泛化能力

- 减少建模特征，提高模型训练与预测速度

##### 处理方法：

- 基于数据理解，直接删除

- 使用主成分分析法（PCA）对特征进行提取

- 使用机器学习模型对特征进行筛选



In [ ]:
# 查看样例数据特征表
df.columns

In [ ]:
# 载入sklearn里的pca模块
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
x = df[['complete_year','area', 'daypop', 'sub_kde',
       'bus_kde', 'kind_kde', 'per_a20_39']]
# 使用x进行pca模型训练同时对x进行转换
x_transform = pca.fit_transform(x)
# 查看主成分对于数据的解释方差百分比
print(pca.explained_variance_ratio_.sum())

In [ ]:
# 查看保留主成分数量与方差解释百分比
for i in range(1,8):
    pca = PCA(n_components=i)
    x_transform = pca.fit_transform(x)
    print(f'components={i},explanined_variance={pca.explained_variance_ratio_.sum()}')

#### 常用判断标准：保留数据的解释方差累计百分比达到95%的所有特征

In [ ]:
# 使用pipeline整合数据标准化、主成分分析与模型
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
# 构建模型工作流
pipe_lm = Pipeline([
        ('sc',StandardScaler()),
        ('pca',PCA(n_components=1)),
        ('lm_regr',LinearRegression())
        ])
print(pipe_lm)

#### 使用模型进行特征筛选：

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
# 准备筛选数据
x = df[['complete_year','area', 'daypop', 'sub_kde',
       'bus_kde', 'kind_kde', 'per_a20_39']]
print(x.shape[1])
y = df[['average_price']]
# 定义筛选模型
lasso_lm = Lasso(alpha = 500)
# 定义特征筛选器
select_m = SelectFromModel(lasso_lm)
# 训练筛选器
select_m.fit(x,y)
# 运行筛选结果
print(select_m.transform(x).shape[1])

In [ ]:
# 构建模型工作流
pipe_lm = Pipeline([
        ('sc',StandardScaler()),
        ('lasso_select',SelectFromModel(lasso_lm)),
        ('lm_regr',LinearRegression())
        ])
print(pipe_lm)

### 3. 特征扩展

##### 处理目的：

- 解决模型欠拟合

- 捕捉自变量与应变量之间的非线性关系

##### 常见处理方法

###### 多项式拓展：

1）假设数据集中包含自变量a、b

2）如果对自变量做多项式二次扩展

3）自变量集从两个变量扩展为5个变量（a、b、a×a、b×b、a×b）

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# 准备筛选数据
x = df[['complete_year','area', 'daypop', 'sub_kde',
       'bus_kde', 'kind_kde', 'per_a20_39']]
print(x.shape[1])
# 定义特征扩展模型并扩展特征
polynomy = PolynomialFeatures(degree=2)
# 训练筛选器
new_x = polynomy.fit_transform(x)
print(new_x.shape[1])

In [ ]:
# 构建模型工作流
pipe_lm = Pipeline([
        ('sc',StandardScaler()),
        ('poly_trans',PolynomialFeatures(degree=2)),
        ('lm_regr',LinearRegression())
        ])
print(pipe_lm)

##                                          谢谢大家观看